In [1]:
#!pip install bing-image-downloader

In [2]:
#!mkdir Images

In [3]:
#from bing_image_downloader import downloader

#downloader.download('sunflower',limit=30,output_dir='Images',adult_filter_off=True)

In [4]:
#downloader.download('Sun',limit=30,output_dir='Images',adult_filter_off=True)

In [5]:
#downloader.download('cone ice cream',limit=30,output_dir='Images',adult_filter_off=True)

In [6]:
import os
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import numpy as np

In [7]:
target = []
images = []
flat_data = []

DATA_DIR = 'Images'
CATEGORIES = ['sunflower','Sun','cone ice cream']


In [8]:
for category in CATEGORIES:
    class_num = CATEGORIES.index(category) #label encoding
    path = os.path.join(DATA_DIR,category)
    for img in os.listdir(path):
        img_array = imread(os.path.join(path,img),pilmode="RGB")
        #print(img_array)
        #plt.imshow(img_array)
        resized_img = resize(img_array,(150,150,3))
        flat_data.append(resized_img.flatten())
        images.append(resized_img)
        target.append(class_num)

In [9]:
flat_data = np.array(flat_data)
images = np.array(images)
target = np.array(target)

In [10]:
#print(flat_data)

In [11]:
#target

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(flat_data,target,test_size=.3,random_state=1)

In [13]:
from sklearn import svm

svc = svm.SVC(probability=True,kernel='rbf',gamma='scale')
svc.fit(x_train,y_train)

SVC(probability=True)

In [14]:
y_pred = svc.predict(x_test)
y_pred

array([1, 0, 1, 1, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 1, 2, 0, 0, 2, 2, 1, 0,
       2, 1, 0, 1, 0])

In [15]:
y_test

array([1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 0, 1, 2, 0, 0, 2, 2, 1, 0,
       2, 1, 1, 1, 1])

In [16]:
from sklearn.metrics import accuracy_score,confusion_matrix 
accuracy_score(y_pred,y_test)

0.8148148148148148

In [17]:
from sklearn.model_selection import cross_val_score
cross_val_score(svc,flat_data,target,cv=10)

array([0.88888889, 1.        , 0.88888889, 0.88888889, 0.88888889,
       0.66666667, 0.77777778, 0.77777778, 0.77777778, 1.        ])

In [18]:
#svc.support_vectors_

In [19]:
confusion_matrix(y_pred,y_test)

array([[ 4,  3,  2],
       [ 0, 11,  0],
       [ 0,  0,  7]], dtype=int64)

In [20]:
import pickle
pickle.dump(svc,open('img_clf.p','wb'))

In [21]:
model = pickle.load(open('img_clf.p','rb'))

In [22]:
#flat_data = []
#url = input('Enter URL:')
#img = imread(url)
#img_resized = resize(img,(150,150,3))
#flat_data.append(img_resized.flatten())
#flat_data = np.array(flat_data)
#plt.imshow(img_resized)
#y_out = model.predict(flat_data)
#y_out = CATEGORIES[y_out[0]]
#y_out

In [23]:
#!pip install gradio

In [24]:
import gradio as gr
def classify(img):
    flat_data = []
    img = resize(img,(150, 150,3))
    flat_data.append(img.flatten())
    flat_data = np.array(flat_data)
    prediction = model.predict(flat_data)
    class_names = ['sunflower','Sun','cone ice cream']
    print(prediction)
    return class_names[prediction[0]]

In [25]:
#set the user uploaded image as the input array
#match same shape as the input shape in the model
im = gr.inputs.Image(image_mode='L', invert_colors=False, source="upload")
#setup the interface
iface = gr.Interface(
    fn = classify, 
    inputs = im, 
    outputs = gr.outputs.Textbox(type="auto", label=None),
)
iface.launch(share=True,debug=True)

Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://27530.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app
